In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import cv2
sys.path.append('..')
sys.path.append('../visualization')
sys.path.append('../acd/util')
sys.path.append('../acd/scores')
sys.path.append('../acd/agglomeration')

import tiling_2d as tiling
import agg_2d as agg
import cd, glob
import viz_2d as viz
import score_funcs
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device", device)
# get dataset
import pickle as pkl
from dsets.imagenet import dset
imnet_dict = pkl.load(open('../dsets/imagenet/imnet_dict.pkl', 'rb')) # contains 6 images (keys: 9, 10, 34, 20, 36, 32)

print("imnet_dict", imnet_dict.keys())
#----
# for idx, t in enumerate(imnet_dict[9]):
#     print("idx", idx, "imnet_dict[9]",type(t), t.shape ,t)
#----
# get model
from torchvision import models
model = models.vgg16(pretrained=True).to(device).eval()
model_type='vgg' # alexnet, vgg

device cuda
imnet_dict dict_keys([32, 34, 20, 9, 10, 36])


In [ ]:
# hyperparameters
num_iters = 5 # number of iterations to agglomerate before merging remaning blobs (fig uses 4)
percentile_include = 95 # values above this percentile will be added at each iteration (fig uses 95)
method = 'cd' # method to rank importance ('cd' works best, 'build_up' or 'occlusion' are simplest)
sweep_dim = 14 # importances are calculated by blocks of sweep_dim x sweep_dim (14 yields good results for imagenet)
# 
# L = glob.glob('/home/yezheng/github/ExtremeNet/images/*.jpg')
L = glob.glob('/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/*.JPEG')
if [] == L:
    L = glob.glob('/Users/yezheng/github/ExtremeNet/images/*.jpg')
L.sort()
print("L", L)
for idx_jpg, key_num in enumerate(L):
    im_orig = cv2.imread(key_num)/255
    
    key_num = key_num.split('/')[-1].split('.')[-2]
    print("========================")
    print("idx_jpg", idx_jpg, "key_num", key_num)
    try:
        im_orig = cv2.resize(im_orig, dsize = (224,224) )

        im_torch = torch.from_numpy(im_orig).permute(2,0,1).float()[None,...]
        print("im_orig", np.max(im_orig), np.min(im_orig),im_orig.shape)
        print("im_torch", im_torch.shape)
        lab_pred = np.argmax(dset.pred_ims(model, np.copy(im_orig)))
        print("key_num",key_num,"lab_pred")
        lists = agg.agglomerate(model, dset.pred_ims, percentile_include, method, sweep_dim, im_orig, 
                                lab_pred, num_iters=num_iters, im_torch=im_torch, model_type=model_type, device = device)        
        print("key_num",key_num,"agg.agglomerate")

        # visualize
        plt.figure(figsize=(12, 5), facecolor='white', dpi=100)
        rows = 3      
        num_ims = len(lists['scores_list'])

        # original plots
        lab_num_correct = 0
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                                     lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                                     subplot_rows=rows, dset=dset, key_num  =key_num)

        # comp plots
        viz.visualize_ims_list(lists['comps_list'],
                              title='Chosen blobs',
                              subplot_row=1, subplot_rows=rows, colorbar=False, im_orig=im_orig, plot_overlay=True, key_num  =key_num)

        # dict plots
        viz.visualize_dict_list_top(lists['comp_scores_raw_list'], method,
                               subplot_row=2, subplot_rows=rows, ind=ind, labs=labs, use_orig_top=True, key_num  =key_num)
    except Exception as e: 
        print("ERROR",e)

  0%|          | 0/256 [00:00<?, ?it/s]

L ['/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_10.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_11.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_12.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_13.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_14.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_15.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_16.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_17.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_18.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_19.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_20.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/images/val_21.JPEG', '/home/yezheng/github/acd/imagenet/tiny-imagenet-200/val/imag

100%|██████████| 256/256 [00:09<00:00, 27.86it/s]
/home/yezheng/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
../acd/agglomeration/agg_2d.py:28: RuntimeWarning: invalid value encountered in greater_equal
  im_thresh = np.logical_and(scores >= thresh, ~np.isnan(scores))
../acd/agglomeration/agg_2d.py:36: RuntimeWarning: invalid value encountered in greater_equal
  im_thresh = np.logical_and(scores >= thresh, ~np.isnan(scores))
  1%|          | 3/256 [00:00<00:08, 28.18it/s]

[agg_2d.py agglomerate] score_funcs.get_scores_2d
[agg_2d.py agglomerate] threshold_scores
Percentiles must be in the range [0, 100]
idx_jpg 1 key_num val_11
im_orig 1.0 0.0 (224, 224, 3)
im_torch torch.Size([1, 3, 224, 224])
key_num val_11 lab_pred
[agg_2d.py agglomerate] im_orig (224, 224, 3)
[agg_2d.py agglomerate] device cuda tiles 256
[score_funcs.py get_scores_2d] device cuda ims (256, 224, 224)


  1%|          | 3/256 [00:00<00:09, 27.75it/s]

...

  1%|          | 3/256 [00:00<00:09, 27.58it/s]

idx_jpg 28 key_num val_38
im_orig 0.9688075670484547 0.0 (224, 224, 3)
im_torch torch.Size([1, 3, 224, 224])
key_num val_38 lab_pred
[agg_2d.py agglomerate] im_orig (224, 224, 3)
[agg_2d.py agglomerate] device cuda tiles 256
[score_funcs.py get_scores_2d] device cuda ims (256, 224, 224)


  1%|          | 3/256 [00:00<00:09, 27.59it/s]

[agg_2d.py agglomerate] score_funcs.get_scores_2d
[agg_2d.py agglomerate] threshold_scores
Percentiles must be in the range [0, 100]
idx_jpg 29 key_num val_39
im_orig 1.0 0.0005402185765192638 (224, 224, 3)
im_torch torch.Size([1, 3, 224, 224])
key_num val_39 lab_pred
[agg_2d.py agglomerate] im_orig (224, 224, 3)
[agg_2d.py agglomerate] device cuda tiles 256
[score_funcs.py get_scores_2d] device cuda ims (256, 224, 224)


  6%|▌         | 15/256 [00:00<00:08, 27.60it/s]

In [ ]:
print("Done!")